In [2]:
import numpy as np

# mne library
import mne
from mne import  read_epochs
from mne.decoding import Vectorizer
mne.set_log_level('error') # Avoid long log

# Sklearn library
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression



In [3]:
# The path to the folder with the data
path = 'files/prepocess_ICA'

# Create a list with each tipe of experimental run
openeye_runs = [1]
closedeye_runs = [2]
fists_runs = [3, 7, 11]
imaginefists_runs = [4, 8, 12]
fistsfeet_runs = [5, 9, 13]
imaginefistsfeet_run = [6, 10, 14]

# List of participants
participants = [i for i in range(1,31)]


# Here we charge the data, the idea is to keep the data of each subject together. 
# So we will create a list that contain a list for each subject, which containts all the epochs of the three experiences.
data = []
labels = []


for participant in participants:
    participant_data = []
    participant_labels = []
    for run in fists_runs:
        epochs = read_epochs(path + f'/S{participant:03}R{run:02}filt01_30_ICA_masref.fif')
        participant_data.append(epochs.get_data())
        participant_labels.append(epochs.events[:,-1])
    data.append(np.concatenate(participant_data, axis = 0))
    labels.append(np.concatenate(participant_labels, axis = 0))

FileNotFoundError: File does not exist: "c:\Users\Stagiaire\Desktop\eeg-TFA-DEVIA\files\prepocess_ICA\S001R03filt01_30_ICA_masref.fif"

In [1]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=0)

X_train = np.concatenate(X_train, axis = 0)
X_test = np.concatenate(X_test, axis = 0)
y_train = np.concatenate(y_train, axis = 0)
y_test = np.concatenate(y_test, axis = 0)

X_train, y_train = shuffle(X_train, y_train, random_state = 42)

print("X_tran dimensions: ", X_train.shape)
print("y_tran dimensions: ", y_train.shape)
print("X_test dimensions: ", X_test.shape)
print("y_test dimensions: ", y_test.shape)

NameError: name 'train_test_split' is not defined

In [85]:
X_train.shape

(1940, 64, 817)

In [86]:
lr_pip = make_pipeline(Vectorizer(), StandardScaler(), LogisticRegression(random_state=42))
parameters = {'logisticregression__solver' : ['newton-cg', 'sag', 'saga', 'lbfgs']}
gs_lr_pip = GridSearchCV(lr_pip, parameters, scoring='accuracy')

In [87]:
gs_lr_pip.fit(X_train, y_train)

/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/elia/anaconda3/envs/mne/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/elia/anaconda3/envs/mne/

GridSearchCV(estimator=Pipeline(steps=[('vectorizer',
                                        <mne.decoding.transformer.Vectorizer object at 0x79dd1b64db20>),
                                       ('standardscaler', StandardScaler()),
                                       ('logisticregression',
                                        LogisticRegression(random_state=42))]),
             param_grid={'logisticregression__solver': ['newton-cg', 'sag',
                                                        'saga', 'lbfgs']},
             scoring='accuracy')

In [88]:
y_pred = gs_lr_pip.predict(X_test)

In [89]:
print(sum(y_test == y_pred))

296


In [90]:
print(sum(y_test == y_pred)/len(y_test))


0.6205450733752621
